In [ ]:
# pip install google-generativeai
# pip install faiss-cpu
# pip install python-telegram-bot

## Embeddings Yaradılması

In [1]:
import google.generativeai as genai
import numpy as np
import faiss
import json
import pickle
import os

from dotenv import load_dotenv

# .env faylından API açarını yükləyək
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# API Key konfiqurasiyası
genai.configure(api_key=GOOGLE_API_KEY)

def create_embeddings():
    with open('labor_law_chunks.json', 'r', encoding='utf-8') as f:
        chunks = json.load(f)

    # Mətnləri və metadatanı ayırırıq
    texts = [item['content'] for item in chunks]
    metadata = [item['metadata'] for item in chunks]
    
    # Google Gemini ilə embedding-lərin yaradılması
    model = 'models/gemini-embedding-001'
    result = genai.embed_content(model=model, content=texts, task_type="RETRIEVAL_DOCUMENT")
    embeddings = result['embedding']

    # FAISS indeksi yaratmaq
    dimension = len(embeddings[0])
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings, dtype=np.float32))

    # Faylların saxlanması
    faiss.write_index(index, 'law_embeddings.faiss')
    with open('chunks_for_retrieval.pkl', 'wb') as f:
        pickle.dump({'texts': texts, 'metadata': metadata}, f)

    print("Embeddings uğurla yaradıldı və saxlandı.")

if __name__ == '__main__':
    create_embeddings()

Embeddings uğurla yaradıldı və saxlandı.
